# Develop superres with Unet
> Develop superres with UNet

In [1]:
#| default_exp preprocessing.superres_with_unet

In [2]:
#| hide
%load_ext autoreload
%autoreload 2


In [3]:
#| export
import sys
from pathlib import Path


In [4]:
#| export
CV_TOOLS = Path(r'/home/ai_sintercra/homes/hasan/projects/git_data/cv_tools')
sys.path.append(str(CV_TOOLS))


In [5]:
#| export
custom_lib_path = Path(r'/home/ai_warstein/homes/goni/custom_libs')
sys.path.append(str(custom_lib_path))


In [6]:
#| export
from cv_tools.imports import *
from cv_tools.core import *
from dotenv import load_dotenv


In [7]:
#| export
load_dotenv(dotenv_path=f'/home/ai_sintercra/Users/goni/workspace/projects/git_data/fastai_course22p2/fastai_course22p2/.env')

False

In [8]:
#| export
CURRETNT_NB='/home/ai_sintercra/Users/goni/workspace/projects/git_data/fastai_course22p2/nbs'

In [34]:
#| export
#----------------------------------
import matplotlib.pyplot as plt
import matplotlib as mpl
from functools import partial
from glob import glob
import shutil

In [27]:
#| export
# fastai fastcore relate
from fastcore.all import *
from fastprogress import progress_bar

In [48]:
#| export
# torch related
import torch
from torch import nn,tensor
from torch.nn import init
from torch.optim import lr_scheduler
from torch import optim
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import torch.nn.functional as F

from torchvision.io import read_image, ImageReadMode
from torcheval.metrics import MulticlassAccuracy

In [29]:
#| export
# huggingface
from datasets import load_dataset


In [62]:
#| export
#  miniai
from miniai.datasets import *
from miniai.conv import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.accel import *
from miniai.training import *
from miniai.augment import *

In [31]:
torch.set_printoptions(
    precision=5,
    linewidth=140,
    sci_mode=False
)
torch.manual_seed(1)
mpl.rcParams['figure.dpi']=70

# Data path

In [32]:
DATA_PATH=os.getenv('DATA_PATH')
tiny_imagenet_root = Path(DATA_PATH, 'tiny-imagenet-200')
tiny_imagenet_root.mkdir(exist_ok=True, parents=True)

In [39]:
#| export
def get_tiny_imagenet_data(
    unzip:bool=True,
    save_path:str=None
    ):
    'Get imagenet tiny data'
    if not save_path:
        save_path = Path('data')
        Path(save_path).mkdir(exist_ok=True, parent=True)
    url= "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    zip_path = Path(save_path, 'tiny-imagenet-200.zip')
    if not zip_path.exists():
        print(f'Downloading {url} to {zip_path}')
        urlsave(url, zip_path)
    if unzip:
        print(f'Unzipping {zip_path} to {save_path}')
        shutil.unpack_archive(zip_path, save_path)

In [36]:
get_tiny_imagenet_data(unzip=True, save_path=tiny_imagenet_root)

Unzipping C:\Users\goni\workspace\projects\data\tiny-imagenet-200\tiny-imagenet-200.zip to C:\Users\goni\workspace\projects\data\tiny-imagenet-200


In [43]:
root_data = Path(tiny_imagenet_root, 'tiny-imagenet-200')

In [55]:
bs = 512
xmean,xstd = (
    tensor([0.47565, 0.40303, 0.31555]), 
    tensor([0.28858, 0.24402, 0.26615]))

tfms = nn.Sequential(
    T.Pad(8),
    T.RandomCrop(64),
    T.RandomHorizontalFlip(),
)



In [51]:
#| export
class TinyDS:
    def __init__(self, root_data:Path):
        self.root_data = root_data
        self.files = glob(
            str(root_data/'**/*.JPEG'),
            recursive=True
        )
    __repr__ = basic_repr('root_data')
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        img = read_image(self.files[idx], ImageReadMode.RGB)/255
        img = (img - xmean[:, None, None])/xstd[:, None, None]
        img = tfms(img)
        return img


In [52]:
#| export
class TfmDS:
    def __init__(
        self, 
        ds, 
        tfmx=noop,
        tfmy=noop):
        store_attr()

    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        item = self.ds[idx]
        x = self.tfmx(item)
        y = self.tfmy(item)
        return x, y
    
    

In [46]:
#| export
def denorm(x):
    return x*xstd[:, None, None] + xmean[:, None, None]


In [49]:
#| export
def tfmx(x, erase=True):
    x =TF.resize(x, (32,32))[None]
    x = F.interpolate(x, scale_factor=2)
    if erase:
        x = rand_erase(x)
    return x[0]





In [58]:
tds = TinyDS(root_data/'train')
vds = TinyDS(root_data/'val')
tds[0].shape

torch.Size([3, 64, 64])

In [59]:
tfm_tds = TfmDS(tds, tfmx=tfmx)
tfm_vds = TfmDS(vds, partial(tfmx, erase=False))

In [63]:
tfm_tds[0][0].shape, tfm_vds[0][0].shape

(torch.Size([3, 64, 64]), torch.Size([3, 64, 64]))

In [66]:
#| export
dls = DataLoaders(*get_dls(
    tfm_tds, 
    tfm_vds, 
    bs=bs,
    num_workers=8))

In [ ]:
xb, yb = next(iter(dls.train))
xb.shape, yb.shape
denorm(xb[0]).shape





In [ ]:
show_images(denorm(xb[:4], im_size=2.5))

In [ ]:
show_images(denorm(yb[:4], im_size=2.5))

# Denoising Autoencoder

In [ ]:
#| export
def up_block(
    ni, 
    nf, 
    ks=3, 
    act=act_gr, 
    norm=None):
    return Sequential(
        nn.UpsampleNearest2d(scale_factor=2),
        ResBlock(ni, nf, ks=ks, act=act_gr, norm=norm),
    )




In [ ]:
#| export
iw = partial(init_weights, leaky=0.1)


In [ ]:
torch.cuda.is_available()

In [ ]:
metrics = MetricsCB()
cbs = [DeviceCB(), metrics, ProgressCB(plot=True), MixedPrecision()]
lr_cbs = [DeviceCB(), ProgressCB(), MixedPrecision()]
opt_func = partial(optim.AdamW, eps=1e-5)

In [ ]:
get_model

In [ ]:
Learner(get_model().apply(iw),dls, F.mse_loss, cbs=lr_cbs,opt_func=opt_func).lr_find(start_lr=1e-4, gamma=1.2)

In [9]:
#| hide
import nbdev; nbdev.nbdev_export('09_preprocessing.superres_with_unet.ipynb')